<a href="https://colab.research.google.com/github/MASSIMOQSELLA/MyAKarpathyMicrogradTest/blob/main/Signature_NN_preelaboration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.metrics import structural_similarity as ssim

#!git clone https://github.com/MASSIMOQSELLA/MyAKarpathyMicrogradTest.git
#!git remote set-url origin https://github.com/MASSIMOQSELLA/MyAKarpathyMicrogradTest.git
#!git config --global user.name "USER"
#!git config --global user.email "email "
#!git remote set-url origin https://<TOKEN>@github.com/MASSIMOQSELLA/MyAKarpathyMicrogradTest.git
#!git remote -v

def img_show(img, title):
# Visualizzare l'immagine
  plt.imshow(img, cmap='gray')
  plt.title(title)
  plt.axis("off")  # Nascondi gli assi
  plt.show()

def img_show_cb(img,title):
  # Visualizza l'immagine normalizzata
  plt.imshow(img, cmap='gray')
  plt.title(f"{title}")
  plt.colorbar()  # Mostra la scala dei valori
  plt.show()

def image_resize_normalize(image):
  image_resized = cv2.resize(image, (56, 56))
  image_normalized = image_resized / 255.0
  return image_normalized

def image_threshold(image):
  #threshold
  _, image_threshold = cv2.threshold(image, 220, 255, cv2.THRESH_BINARY)
  return image_threshold

def rotate_image(image, angle):
  """Ruota un'immagine di un dato angolo."""
  (h, w) = image.shape[:2]
  center = (w // 2, h // 2)
  # Calcola la matrice di rotazione
  rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
  # Applica la rotazione
  rotated = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
  return rotated

def translate_image(image, x, y):
    """Sposta l'immagine di x e y pixel.tenta con 10, -5"""
    translation_matrix = np.float32([[1, 0, x], [0, 1, y]])
    shifted = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return shifted

def scale_image(image, scale_factor):
    """Scala un'immagine con un dato fattore. prova con 1.2"""
    height, width = image.shape[:2]
    scaled = cv2.resize(image, (int(width * scale_factor), int(height * scale_factor)), interpolation=cv2.INTER_LINEAR)
    return scaled

def affine_transform(image):
    """Applica una distorsione affine all'immagine."""
    rows, cols = image.shape[:2]
    # Punti originali
    src_points = np.float32([[0, 0], [cols-1, 0], [0, rows-1]])
    # Punti trasformati
    dst_points = np.float32([[0, 0], [cols-1, 0], [int(0.5*cols), rows-1]])
    # Matrice affine
    affine_matrix = cv2.getAffineTransform(src_points, dst_points)
    distorted = cv2.warpAffine(image, affine_matrix, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return distorted

def add_noise(image):
    """Aggiunge rumore all'immagine."""
    noise = np.random.normal(0, 15, image.shape).astype(np.uint8)  # Rumore gaussiano
    noisy_image = cv2.add(image, noise)
    return noisy_image

def image_to_vector(image):
    # Appiattisce l'immagine (da 2D a 1D)
    return image.flatten()

def vector_to_image(vector, shape = [56, 56]):
    # Ripristina l'immagine da un vettore
    return vector.reshape(shape)

dataset = []
dir_path = "/content/MyAKarpathyMicrogradTest/signatures_false/"
os.chdir(dir_path)

for image_file in os.listdir(dir_path):

  image_path = os.path.join(dir_path, image_file)

  if(image_file.startswith("sigla_false_")):
    #print (image_path)
    #answ = os.path.exists(image_path)
    #print(answ)

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    #img_show(image, "Original Image")

    img_thr = image_threshold(image)
    img_thr_rsz_nrm  = image_resize_normalize(img_thr)
    #img_show(img_thr_rsz_nrm, "Threshold Image resized normalized")
    #img_show_cb(img_thr_rsz_nrm, "cb Threshold Image resized normalized")
    img_thr_vec = image_to_vector(img_thr_rsz_nrm)
    #image_2d = img_thr_vec.reshape(56, 56)  # Ricostruzione in 2D
    #img_show(image_2d, "Threshold Image 2D")
    dataset.append(img_thr_vec)

    img_rot = rotate_image(img_thr, 15)
    img_thr_rot_rsz_nrm  = image_resize_normalize(img_rot)
    img_rot_vec = image_to_vector(img_thr_rot_rsz_nrm)
    dataset.append(img_rot_vec)

    img_tra = translate_image(img_thr, 10, -5)
    img_thr_tra_rsz_nrm = image_resize_normalize(img_tra)
    img_tra_vec = image_to_vector(img_thr_tra_rsz_nrm)
    dataset.append(img_tra_vec)

    img_sca = scale_image(img_thr, 1.2)
    img_thr_sca_rsz_nrm = image_resize_normalize(img_sca)
    img_sca_vec = image_to_vector(img_thr_sca_rsz_nrm)
    dataset.append(img_sca_vec)

    img_aff = affine_transform(img_thr)
    img_thr_aff_rsz_nrm = image_resize_normalize(img_aff)
    img_aff_vec = image_to_vector(img_thr_aff_rsz_nrm)
    dataset.append(img_aff_vec)

    img_noi = add_noise(img_thr)
    img_thr_noi_rsz_nrm = image_resize_normalize(img_noi)
    img_noi_vec = image_to_vector(img_thr_noi_rsz_nrm)
    dataset.append(img_noi_vec)
%cd /content/MyAKarpathyMicrogradTest/signatures_false/
dataset = np.array(dataset)
print('writing dataset with dataset dimensions: ', dataset.shape)
np.save('dataset_sigle_false_vectors.npy', dataset)

#!git add .
#!git commit -m "Aggiornamento file"
#!git push origin main



/content/MyAKarpathyMicrogradTest/signatures_false
writing dataset with dataset dimensions:  (528, 3136)
[main 096849c] Aggiornamento file
 1 file changed, 0 insertions(+), 0 deletions(-)
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 257.70 KiB | 622.00 KiB/s, done.
Total 8 (delta 6), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 3 local objects.
To https://github.com/MASSIMOQSELLA/MyAKarpathyMicrogradTest.git
   0ad9459..096849c  main -> main
